# Generate protected-status report

In [ ]:
import hubmapbags
from datetime import datetime
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from warnings import warn as warning

token = ''
instance = 'prod' #default instance is test

In [ ]:
# get assay types
assay_names = hubmapbags.get_assay_types()

In [ ]:
report = pd.DataFrame()
for assay_name in assay_names:
    print(assay_name)
    datasets = pd.DataFrame(hubmapbags.get_hubmap_ids( assay_name=assay_name, token=token ))

    if datasets.empty:
        continue
    
    #clean up
    datasets = datasets[(datasets['data_type'] != 'image_pyramid')]
    datasets = datasets[(datasets['status'] == 'Published')]
    
    datasets['is_protected'] = None
    for index, datum in tqdm(datasets.iterrows()):
        datasets.loc[index,'is_protected'] = hubmapbags.apis.is_protected( datum['hubmap_id'], instance='prod', token=token )
    
    if report.empty:
        report = datasets
    else:
        report = pd.concat( [report, datasets] )

In [ ]:
from datetime import datetime
now = datetime.now() 

if not Path('protected-data-report').exists():
    Path('protected-data-report').mkdir()
report.to_csv('protected-data-report/' + str(now.strftime('%Y%m%d')) + '.tsv', sep='\t', index=False )